# MOVIE RECOMMENDATION SISTEM


Recommendation engine je skup razlicitih algoritama koji analiziraju i filtriraju podatke te najrelevantije pokazuje korisnicama. Prvo zabiljezi posljednje postupke korisnika pa na osnovu toga predlaze neke druge ponude. 

Na primjer, ukoliko korisnik često gleda filmove određenog žanra ili kupuje proizvode određene vrste, sistem će zabeležiti ove aktivnosti. Na osnovu tih podataka, recommendation engine može preporučiti slične filmove ili proizvode koji bi mogli biti od interesa korisniku. 



Prvo importujemo potrebne biblioteke i dodajemo imena fajlova koje cemo koristiti u daljem radu.

U konstante zapisujemo putanje do fajlova koje cemo koristiti u daljem radu.

- **movies.csv** - sadrzi podatke o filmovima
- **ratings.csv** - sadrzi podatke o ocjenama filmova
- **credits.csv** - sadrzi podatke o glumcima i ekipi filma

In [1]:
import pandas as pd
import numpy as np

TMDB_MOVIES = "./data/movies.csv"
TMDB_CREDITS = "./data/credits.csv"
RATINGS_SMALL = "./data/ratings.csv"

## 1. Za parsiranje fajlova koristimo pandas i funkciju:

Funckija **read_csv()** iz pandas biblioteke nam omogucava da ucitamo csv fajlove u dataframe. **Columns** je opcioni parametar koji nam omogucava da odaberemo samo odredjene kolone koje zelimo da ucitamo u memoriju.

In [63]:
def parse_csv(filepath: str, columns: list = None) -> pd.DataFrame:
    if columns is None:
        df = pd.read_csv(filepath)
    else:
        df = pd.read_csv(filepath, usecols=columns)
    return df

Kako imamo 3 fajla koja cemo koristiti, prvo cemo ih ucitati u memoriju, a zatim biramo kolone koje su nam potrebne za dalji rad. Pa tako spajamo podatke iz fajlova **movies.csv** i **credits.csv** u jedan dataframe. I koristimo kolone:
- **id** - id filma
- **title** - naziv filma
- **cast** - glumci
- **crew** - ekipa filma
- **keywords** - kljucne rijeci filma
- **genres** - zanrovi filma
- **vote_average** - prosjecna ocjena filma
- **vote_count** - broj ocjena filma
- **overview** - kratak opis filma


In [65]:
# number of first n movies filters will return
RESULT_SIZE = 5
attributes = ["movie_id", "title", "cast", "crew"]
df_credits = parse_csv(TMDB_CREDITS, attributes)
# changing the name of cols
df_credits.columns = ["id", "title", "cast", "crew"]

# df movies
attributes = ["genres", "id", "keywords", "vote_average", "vote_count", "tagline", "overview"]
df_movies = parse_csv(TMDB_MOVIES, attributes)

df_movies = df_movies.merge(df_credits, on="id")
df_movies.head()

,genres,id,keywords,overview,tagline,vote_average,vote_count,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Legend Ends,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","Lost in our world, found in another.",6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Nakon uspjesnog spajanja fajlova dobijamo potrebnu tabelu koju mozemo koristiti u nastavku.

## 2. Demografsko filtriranje
Ovi sistemi pružaju generalizovane preporuke svakom korisniku na osnovu popularnosti filma i/ili žanra. Osnovna ideja iza ovog sistema jeste da će filmovi koji su popularni i dobili kritičke pohvale imati veću verovatnoću da se sviđaju prosečnoj publici.


Metrika koju ćemo koristiti za sortiranje filmova je IMDB-ov weighted rating:


![wr.png](images/wr.png)

- **v** - je broj glasova za film
- **m** - je minimalni broj glasova potreban da bi se našao na listi
- **R** - je prosječna ocjena filma
- **C** - je prosječni broj glasova na listi


In [66]:
CORRECT_FACTOR = 100000
def rate_movie(movie, m, C):
    v = movie["vote_count"]
    R = movie["vote_average"]
    return ((v/(v+m)*R) + m*(m+v)*C)/CORRECT_FACTOR

In [67]:
# filter the given dataset and returns top resultsize movies
# wr = (v/(v+m)*R) + (m/(m+v)*C)
# v is the number of votes for the movie;
# m is the minimum votes required to be listed in the chart;
# R is the average rating of the movie; And
# C is the mean vote across the whole report
def demographic_filtering(movies: pd.DataFrame, result_size: int = 20):
    # v - vote_count
    # R - vote_average
    m = movies["vote_count"].quantile(0.9)
    C = movies["vote_average"].mean()
    

    # removing all movies with low number of votes
    movies = movies.loc[movies["vote_count"] >= m]
    # rating every movie
    rate = movies.apply(lambda movie: rate_movie(movie, m, C), axis=1)
    movies.insert(0, "rate", rate)
    # sort my resulted movies by rate
    movies = movies.sort_values(by="rate", ascending=False)
    return movies[0:result_size]

Na osnovu ove metrike mozemo izracunati tezinu svakog filma i sortirati ih po toj tezini, te tako dobiti najbolje ocijenjene filmove za metodu demografskog filtriranja.

Vracamo fiksan broj najboljih filmova koji je odredjen parametrom RESLULT_SIZE.

In [68]:
results = demographic_filtering(df_movies, RESULT_SIZE)
results[["rate","title", "vote_count", "vote_average"]].head(RESULT_SIZE)

,rate,title,vote_count,vote_average
96,1746.101204,Inception,13752,8.1
65,1550.103861,The Dark Knight,12002,8.2
0,1527.480159,Avatar,11800,7.2
16,1524.792197,The Avengers,11776,7.4
788,1437.321382,Deadpool,10995,7.4


## 3. Kolaorativno filtriranje

Ovaj sistem upoređuje osobe sa sličnim interesovanjima i pruža preporuke na osnovu ovog podudaranja. Kolaborativni filteri ne zahtijevaju metapodatke elementa (filmova) kao njihovi sadržajno bazirani pandani.

Postoje dvije vrste kolaborativnog filtriranja: user-based i item-based. Implementiraćemo oba i uporediti rezultate.

Koristićemo **Single Value Decomposition** da smanjimo dimenzionalnost problema. 
Koristićemo metrike poput **RMSE** i **MSE** za upoređivanje rezultata.


- postavljamo imena kolona koje cemo koristiti u daljem radu
- mijenjamo ime kolone movieId u id da bi mogli da spojimo tabele
- citamo fajlove
- spajamo tabele movies i ratings

In [69]:
# COLLABORATIVE FILTERING
attributes = ["userId", "movieId", "rating"]
# loading the files
df_movies = df_movies.rename(columns={"id": "movieId"})

df_ratings = parse_csv(RATINGS_SMALL, attributes)

titles = df_movies[["title", "movieId"]]
# merging two tables
df_ratings_title = df_ratings.merge(titles, on="movieId")
df_ratings_title.head(10)

,userId,movieId,rating,title
0,1,2105,4.0,American Pie
1,4,2105,4.0,American Pie
2,15,2105,4.0,American Pie
3,30,2105,2.0,American Pie
4,34,2105,4.0,American Pie
5,35,2105,3.5,American Pie
6,41,2105,5.0,American Pie
7,49,2105,3.0,American Pie
8,59,2105,1.0,American Pie
9,73,2105,3.0,American Pie


Pogledajmo kako izgleda nasa matrica korisnika i filmova. Koristicemo funkciju **get_user_item_matrix** koja vraca matricu korisnika i filmova. Na x-osi su filmovi, a na y-osi korisnici. Vrijednosti u matrici su ocjene koje su korisnici dali filmovima.

In [6]:
def get_user_item_matrix(df: pd.DataFrame) -> pd.DataFrame:
    user_item_matrix = df.pivot(
        index=["userId"], columns=["movieId"], values=["rating"]
    ).fillna(0)
    return user_item_matrix


In [70]:
get_user_item_matrix(df_ratings_title)

rating                                                                 \
movieId 5      11     12     13     14     16     18     19     20     22       
userId                                                                          
1          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
667        0.0    3.0    0.0    0.0    0.0    2.0    0.0    0.0    0.0    0.0   
668        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
669        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
670        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
671        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

         ...                                                                 \
movieId  ... 64499  69640  74510  77866  89492  98369  103731 114635 115210   
userId   ...                                                                  
1        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
667      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
668      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
669      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
670      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
671      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                
movieId 116977  
userId          
1          0.0  
2          0.0  
3          0.0  
4          0.0  
5          0.0  
...        ...  
667        0.0  
668        0.0  
669        0.0  
670        0.0  
671        0.0  

[670 rows x 857 columns]

Kao sto smo mogli pretpostaviti matrica je veoma rijetka, jer svaki korisnik je ocijenio samo mali broj filmova. To je veliki problem za kolaborativno filtriranje, jer ne mozemo da izracunamo slicnost izmedju korisnika koji nisu ocijenili iste filmove. 

Zbog toga cemo koristiti **SVD** da smanjimo dimenzionalnost problema.

## Sta je SVD (Single Value Decomposition)?

SVD je tehnika za smanjivanje dimenzionalnosti koja se koristi za kompresiju podataka. SVD se može koristiti za smanjivanje dimenzionalnosti u skupu podataka. 

Jedan način za rukovanje skalabilnošću i retkoćom koju stvara kolaborativno filtriranje je iskorišćavanje latentnog faktorskog modela kako bi se uhvatila sličnost između korisnika i filmova. Bitno je pretvoriti problem recommendation sistema u problem optimizacije. Možemo ga posmatrati kao koliko smo dobri u predviđanju ocena za filmove datog korisnika. Jedna uobičajena metrika je Koren srednjekvadratna greška (RMSE). Što je niži RMSE, to je bolja performansa.

Sada, kada govorimo o latentnom faktoru, možda se pitate šta je to? To je široka ideja koja opisuje osobinu ili koncept koji korisnik ili stavka imaju. Na primer, za muziku, latentni faktor može se odnositi na žanr kojem muzika pripada. SVD smanjuje dimenziju matrice korisnosti izvlačeći njene latentne faktore. Bitno je mapirati svakog korisnika i svaku stavku u latentni prostor sa dimenzijom r. Na taj način, pomaže nam da bolje razumemo odnos između korisnika i stavki jer postaju direktno uporedivi. Sledeća slika ilustruje ovu ideju.

![wr.png](images/SVD-1.jpg.webp)



![wr.png](images/svd_graph.png)

- Prvo cemo importovati SVD iz surprise biblioteke i sve potrebne funkcije

In [7]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split

### 1.1 User-based kolaborativno filtriranje

Ovi sistemi preporučuju proizvode korisniku koje su slični korisnici voljeli. Za mjerenje sličnosti između dva korisnika možemo koristiti kosinusnu sličnost. Ova tehnika filtriranja može se ilustrovati primerom. 

U sledećim matricama, svaki red predstavlja korisnika, dok kolone odgovaraju različitim filmovima osim poslednje koja beleži sličnost između tog korisnika i ciljnog korisnika. Svaka ćelija predstavlja ocjenu koju je korisnik dao tom filmu. Pretpostavimo da je korisnik E ciljni korisnik.

![wr.png](images/ub.png)

Prvo cemo odrediti slicnost izmedju korisnika. Pa pomocu te slicnosti cemo izracunati ocjenu koju bi ciljni korisnik dao filmu.

Tako dobijene filmove sortiramo po ocjeni i vracamo najbolje ocijenjene filmove.

- Treniramo nas model na trening setu

In [71]:
def get_svd_model(df: pd.DataFrame) -> SVD:
    reader = Reader()
    # convert pandas dataframe to surprise dataset
    data = Dataset.load_from_df(df, reader)

    svd = SVD()
    # Perform train-test split
    trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

    svd.fit(trainset)

    predictions = svd.test(testset)
    
    # Then compute RMSE
    accuracy.rmse(predictions)

    return svd

Za odredjenog korisnika mozemo dobiti njegove najbolje preporuke. Izracunavamo slicnost izmedju korisnika i sortiramo ih po slicnosti. Vracamo fiksni broj najboljih preporuka. 

Takodje mozemo koristiti funkciju **resize_movie_dataset** da bi suzili skup filmova na one koji su najbolje ocijenjeni.

In [72]:
# Resize movie dataset to get most popular movies to apply collaborativve filtering on them
def resize_movie_dataset(movie_df: pd.DataFrame, procentage: float = 0.7):
    m = movie_df["vote_count"].quantile(procentage)
    movie_df = movie_df.loc[movie_df["vote_count"] >= m]
    return movie_df

In [73]:
def CF_get_top_movies_for_user(
    user_id: int,
    df_users: pd.DataFrame,
    df_movies: pd.DataFrame,
    result_size: int = 20,
):
    # movies = resize_movie_dataset(df_movies, 0.9)
    movies = df_movies
    model = get_svd_model(df_users)
    # print(movies.head(5))
    # print(movies.columns)
    # rating every movie
    rate = movies.apply(
        lambda movie: model.predict(user_id, movie["movieId"])[3], axis=1
    )
    movies.insert(0, "rate", rate)
    # print(movies.head(5))
    movies = movies.sort_values(by="rate", ascending=False)
    return movies[0:result_size]

Provjeravamo korisnika sa odredjenim id - jem kako bi vidjeli koje filmove je ocijenio i koje filmove mu preporucujemo.

In [96]:
titles = df_movies[["title", "movieId"]]
# merging two tables
df_ratings_title = df_ratings.merge(titles, on="movieId")
user_ratings = df_ratings_title.loc[df_ratings_title["userId"] == 5]
# user_ratings
titles.shape
df_ratings_title.shape
user_ratings.sort_values(by="rating", ascending=False).head(10)

,userId,movieId,rating,title
7308,5,597,5.0,Titanic
7903,5,4995,4.5,Boogie Nights
2461,5,500,4.5,Reservoir Dogs
7273,5,277,4.5,Underworld
6333,5,1961,4.0,My Name Is Bruce
7523,5,1777,4.0,Fahrenheit 9/11
7021,5,104,4.0,Run Lola Run
1605,5,364,4.0,Batman Returns
48,5,2294,4.0,Jay and Silent Bob Strike Back
5240,5,440,4.0,Aliens vs Predator: Requiem


In [62]:
if 'rate' in df_movies.columns:
    df_movies = df_movies.drop(columns=['rate'], axis=1)
results = CF_get_top_movies_for_user(99, df_ratings,df_movies, RESULT_SIZE)
results[["rate", "title", "vote_count", "vote_average"]]

RMSE: 0.9060


,rate,title,vote_count,vote_average
1666,4.272616,The Good Thief,31,6.0
1145,4.264037,The Sixth Sense,3147,7.7
150,4.184722,Men in Black II,3114,6.0
2572,4.184616,Boogie Nights,796,7.5
2108,4.126011,Edward Scissorhands,3601,7.5
509,4.073650,Madagascar,3237,6.6
448,4.015478,Cold Mountain,533,6.7
1025,3.996926,The Thomas Crown Affair,339,6.6
1260,3.953097,Amélie,3310,7.8
217,3.921087,Ghost Rider,1712,5.2


### 1.2 Item-based kolaborativno filtriranje

- Treniramo nas model na trening setu
- Za odredjeni film mozemo dobiti njegove najbolje preporuke. Izracunavamo slicnost izmedju filmova i sortiramo ih po slicnosti. Vracamo fiksni broj najboljih preporuka.

- Prvo za odredjeno ime filma dobijamo njegov id

In [39]:
def get_movie_id(movieName: str, movies: pd.DataFrame):
    movie = movies.loc[movies["title"] == movieName]
    # if we have more than one movie with the same name but they are different
    # i will just send the first because later we will send id with each movie!
    if len(movie) > 1:
        print(movie.iloc[0]["movieId"])
        return movie.iloc[0]["movieId"]
    
    elif movie.empty:
        return None
    try:
        movie_id = int(movie["movieId"])
        return movie_id
    except ValueError:
        return None

- Zatim za taj id dobijamo preporuke

In [32]:
def get_svd_model_ub(df: pd.DataFrame) -> SVD:
    reader = Reader()
    # convert pandas dataframe to surprise dataset
    data = Dataset.load_from_df(df[['movieId', 'movieId', 'rating']], reader)

    svd = SVD()
    # Perform train-test split
    trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

    svd.fit(trainset)

    predictions = svd.test(testset)
    
    # Then compute RMSE
    accuracy.rmse(predictions)

    return svd

In [33]:
def CF_get_top_movies_for_movie(movieName:str, 
                                df_users: pd.DataFrame, 
                                df_movies: pd.DataFrame,
                                result_size: int = 20):
    movie_id = get_movie_id(movieName, df_movies)
    if movie_id is None:
        print("Movie doesnt exist")
        return None
    model = get_svd_model_ub(df_users)
    rate = df_movies.apply(
        lambda movie: model.predict(movie_id, movie['movieId'])[3], axis=1
        )
    df_movies.insert(0, 'rate', rate)
    df_movies = df_movies.sort_values(by="rate", ascending=False)
    return df_movies[0:result_size]

In [35]:
if 'rate' in df_movies.columns:
    df_movies = df_movies.drop(columns=['rate'], axis=1)
user_item_matrix = get_user_item_matrix(df_ratings_title)
results = CF_get_top_movies_for_movie("Batman", df_ratings, df_movies, 10)
if not results.empty:
    # user_item_matrix.transpose
    print(results[["rate", "title", "vote_count", "vote_average"]])

268
RMSE: 0.9833
          rate                    title  vote_count  vote_average
2914  4.186660           Doctor Zhivago         247           7.4
1359  4.131567                   Batman        2096           7.0
878   4.110973     Flags of Our Fathers         526           6.7
1782  4.091241         Kindergarten Cop         628           5.8
2569  4.088127              Match Point        1105           7.3
1666  4.051372           The Good Thief          31           6.0
1170  4.049635  The Talented Mr. Ripley         767           7.0
1025  4.033013  The Thomas Crown Affair         339           6.6
425   3.998751      Mission: Impossible        2631           6.7
315   3.992125        The Mummy Returns        2206           6.0
